In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
import time
import random
import os

In [3]:
def generate_date_list(start, end):
    date_list = []
    curr = start
    while curr <= end:
        date_list.append(curr.strftime("%Y.%m.%d"))
        curr += timedelta(days=1)
    return date_list

In [4]:
def extract_urls(json_data):
    urls = []
    for item in json_data.get('collection', []):
        html_str = item.get('html', '')
        soup = BeautifulSoup(html_str, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            if 'n.news.naver.com' in link['href']:
                urls.append(link['href'])
    unique_urls = list(set(urls))
    return unique_urls

In [5]:
def get_urls_bydate(date, office_id):
    collected_urls = []
    base_url = "https://s.search.naver.com/p/newssearch/3/api/tab/more"
    headers = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://search.naver.com/'}
    for start in range(1, 2000, 10):
        params = {
        'abt': 'null',
        'de': date,
        'ds': date,
        'field': '0',
        'is_dts': '0',
        'is_sug_officeid': '0',
        'mynews': '1',
        'news_office_checked': office_id,
        'nqx_theme': '{"theme":{"sub":[{"name":"finance"}]}}',
        'nso': f'so:r,p:from{date.replace(".","")}to{date.replace(".","")},a:all',
        'office_category': '0',
        'office_section_code': '3',
        'office_type': '1',
        'pd': '3',
        'photo': '0',
        'query': '금리',
        'rev': '0',
        'service_area': '0',
        'sm': 'tab_smr',
        'sort': '2',
        'spq': '0',
        'ssc': 'tab.news.all',
        'start': start
        }
        try:
            res = requests.get(base_url, headers=headers, params=params, timeout=10)
            if res.status_code != 200 or not res.text.strip():
                print(f"[{date}] 수집 종료")
                break
            data = res.json()
            page_urls = extract_urls(data)
            
            if not page_urls:
                break
            
            collected_urls.extend(page_urls)
            
            time.sleep(random.uniform(0.3, 0.6))
            
        except Exception as e:
            print(f"[{date}] 에러 발생: {e}")
            raise e
            
    return list(set(collected_urls))

In [6]:
offices = {"매일경제": "1009", "한국경제": "1015", "머니투데이": "1008"}
keyword = "금리"
s_dt = datetime(2005, 5, 1) 
e_dt = datetime(2005, 5, 30)
max_workers = 10

In [7]:
for office in offices:
    office_id = offices[office]
    file_name = f'news_urls_{office}.csv'
    target_dates = generate_date_list(s_dt, e_dt)
    print(f"{office} 기사 총 {len(target_dates)}일치 수집을 시작합니다.")

    for i, date in enumerate(target_dates):
        print(f"\n{office} [진행도: {i+1}/{len(target_dates)}] {date} 수집 중...")
        urls = get_urls_bydate(date, office_id)

        if urls:
            df = pd.DataFrame(urls, columns=['url'])
            df['date'] = date
            if not os.path.exists(file_name):
                df.to_csv(file_name, index=False, encoding='utf-8-sig')
            else:
                df.to_csv(file_name, index=False, encoding='utf-8-sig', mode='a', header=False)
            print(f"{date} 저장 완료({len(urls)}개 추가됨)")
        else:
            print(f"{date}는 검색 결과 없음. 다음 날로 넘어갑니다.")

    print(f"\n모든 수집 완료. '{file_name}' 파일을 확인해 보세요.")


매일경제 기사 총 30일치 수집을 시작합니다.

[진행도: 1/30] 2005.05.01 수집 중...
2005.05.01 저장 완료(8개 추가됨)

[진행도: 2/30] 2005.05.02 수집 중...
2005.05.02 저장 완료(8개 추가됨)

[진행도: 3/30] 2005.05.03 수집 중...
2005.05.03 저장 완료(6개 추가됨)

[진행도: 4/30] 2005.05.04 수집 중...
2005.05.04 저장 완료(8개 추가됨)

[진행도: 5/30] 2005.05.05 수집 중...
2005.05.05 저장 완료(3개 추가됨)

[진행도: 6/30] 2005.05.06 수집 중...
2005.05.06 저장 완료(6개 추가됨)

[진행도: 7/30] 2005.05.07 수집 중...
2005.05.07는 검색 결과 없음. 다음 날로 넘어갑니다.

[진행도: 8/30] 2005.05.08 수집 중...
2005.05.08 저장 완료(5개 추가됨)

[진행도: 9/30] 2005.05.09 수집 중...
2005.05.09 저장 완료(12개 추가됨)

[진행도: 10/30] 2005.05.10 수집 중...
2005.05.10 저장 완료(7개 추가됨)

[진행도: 11/30] 2005.05.11 수집 중...
2005.05.11 저장 완료(12개 추가됨)

[진행도: 12/30] 2005.05.12 수집 중...
2005.05.12 저장 완료(10개 추가됨)

[진행도: 13/30] 2005.05.13 수집 중...
2005.05.13 저장 완료(3개 추가됨)

[진행도: 14/30] 2005.05.14 수집 중...
2005.05.14 저장 완료(1개 추가됨)

[진행도: 15/30] 2005.05.15 수집 중...
2005.05.15 저장 완료(6개 추가됨)

[진행도: 16/30] 2005.05.16 수집 중...
2005.05.16 저장 완료(8개 추가됨)

[진행도: 17/30] 2005.05.17 수집 중...
2005.05.1

In [8]:
# 수집 후 겹치는 url 지우기
for office in offices:
    df = pd.read_csv(f'news_urls_{office}.csv')
    df_clean = df.drop_duplicates(subset=['url'], keep='first')
    df_clean.to_csv(f"news_urls_{office}_final.csv", index=False, encoding='utf-8-sig')

    print(f"정리 전: {len(df)}개 -> 정리 후: {len(df_clean)}개")

정리 전: 365개 -> 정리 후: 179개
정리 전: 275개 -> 정리 후: 275개
정리 전: 683개 -> 정리 후: 683개


In [9]:
# 뉴스 본문 수집
def news_content(url):
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Referer': 'https://news.naver.com/'
    }
    try:
        res = requests.get(url, headers=headers, timeout=15)
        if res.status_code != 200:
            return None
        soup = BeautifulSoup(res.text, 'html.parser')
        
        title = soup.select_one("h2#title_area")
        title = title.get_text(strip=True) if title else "제목 없음"
        
        date_tag = soup.select_one("span.media_end_head_info_datestamp_time")
        if date_tag and date_tag.has_attr('data-date-time'):
            date_str = date_tag['data-date-time']
        else:
            date_str = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
        content = soup.select_one("#newsct_article") # 또는 "#dic_area"
        
        if content:
            content = content.get_text(" ", strip=True)
        else:
            content = "본문 없음"
            
        return {'title': title, 'date': date_str, 'content': content, 'url': url}

    except Exception as e:
        print(f"\n[{url}] 수집 중 에러 발생: {e}") 
        raise e

In [10]:
for office in offices:
    input_file = f"news_urls_{office}_final.csv"
    output_file = f"news_contents_{office}.csv"
    max_workers = 4

    df_input = pd.read_csv(input_file)
    full_urls = set(df_input['url'].tolist())

    already_done_urls = set()
    if os.path.exists(output_file):
        df_existing = pd.read_csv(output_file)
        already_done_urls = set(df_existing['url'].tolist())
        print(f"기존 파일에서 {len(already_done_urls)}개의 수집 완료 데이터를 찾았습니다.")

    urls_to_crawl = [u for u in full_urls if u not in already_done_urls]
    print(f"수집해야 할 기사는 {len(urls_to_crawl)}개입니다.")


    results = []
    # 병렬 처리
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(news_content, url): url for url in urls_to_crawl}
        for i, future in enumerate(as_completed(future_to_url)):
            try:
                data = future.result()
                if data:
                    results.append(data)
            except Exception as e:
                print(f"❌ 에러 발생: {e}")
            # 100개마다 중간 저장
            if (i + 1) % 100 == 0:
                pd.DataFrame(results).to_csv(output_file, index=False, encoding='utf-8-sig', 
                                            mode='a', header=not os.path.exists(output_file))
                results = [] # 저장 후 메모리 비우기
                print(f"✅ {i + 1}개 완료 및 중간 저장됨...")
                
                # 차단 방지
                time.sleep(1)

    # 남은 데이터 저장
    if results:
        pd.DataFrame(results).to_csv(output_file, index=False, encoding='utf-8-sig', 
                                    mode='a', header=not os.path.exists(output_file))

    print(f"\n🎉 모두 수집 완료! '{output_file}'을 확인하세요.")

기존 파일에서 179개의 수집 완료 데이터를 찾았습니다.
수집해야 할 기사는 0개입니다.

🎉 모두 수집 완료! 'news_contents_매일경제.csv'을 확인하세요.
수집해야 할 기사는 275개입니다.
✅ 100개 완료 및 중간 저장됨...
✅ 200개 완료 및 중간 저장됨...

🎉 모두 수집 완료! 'news_contents_한국경제.csv'을 확인하세요.
수집해야 할 기사는 683개입니다.
✅ 100개 완료 및 중간 저장됨...
✅ 200개 완료 및 중간 저장됨...
✅ 300개 완료 및 중간 저장됨...
✅ 400개 완료 및 중간 저장됨...
✅ 500개 완료 및 중간 저장됨...
✅ 600개 완료 및 중간 저장됨...

🎉 모두 수집 완료! 'news_contents_머니투데이.csv'을 확인하세요.
